In [ ]:
!pip install nltk transformers

In [ ]:
import pandas as pd
import os
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from typing import List
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
import itertools
import pickle
import tensorflow as tf
import torch
from transformers import BertModel, BertTokenizerFast

In [ ]:
input_filename = 'items_titles.csv'#os.getcwd().split('/examples/')[0]+'/data/initial_data/items_titles.csv'

df_titles = pd.read_csv(input_filename)

In [ ]:
df_titles.tail()

,ITE_ITEM_TITLE
29995,Tênis Vans Old Skool I Love My Vans - Usado - Feminino
29996,Tênis Feminino Preto Moleca 5296155
29997,Tenis Botinha Com Pelo Via Marte Original Lançamento
29998,Tênis Slip On Feminino Masculino Original Sapato Xadrez Mule
29999,Bicicleta Nathor Rosa Infantil Sem Pedal Balance Aro 12


In [ ]:
df_titles['ITE_ITEM_TITLE'] = df_titles['ITE_ITEM_TITLE'].str.lower()

In [ ]:


tokenizer = BertTokenizerFast.from_pretrained("setu4993/LaBSE")
model = BertModel.from_pretrained("setu4993/LaBSE")
model = model.eval()

In [ ]:
i_row = 0
batch_size = 600
list_sentences = list(df_titles['ITE_ITEM_TITLE'])
list_embeddings = list()
while i_row < df_titles.shape[0]:
  bacth_sentences = list_sentences[i_row : i_row+batch_size]

  tokenized_inputs = tokenizer(bacth_sentences, return_tensors="pt", padding=True)

  with torch.no_grad():
      embedding_out = model(**tokenized_inputs)

  list_embeddings.append( embedding_out )
  i_row = i_row + batch_size

In [ ]:
final_embeddings = [each_embedding.pooler_output for each_embedding in list_embeddings]

In [ ]:
final_embeddings_flat = list(itertools.chain(*final_embeddings))

In [ ]:
tensor_embedding = tf.stack(final_embeddings_flat)

In [ ]:
with open('tensor_embeddings.pkl', 'wb') as handle:
    pickle.dump(tensor_embedding, handle)

In [ ]:
with open('tensor_embeddings.pkl', 'rb') as handle:
    tensor_embedding = pickle.load(handle)

In [ ]:
X_features = np.array(tensor_embedding)

In [ ]:
similarity_dict = {}
for i in tqdm(range(df_titles.shape[0])):
    vec_i = X_features[i:i+1,:]
    vec_j = X_features[i+1:,:]
    similarity_dict[i] = cosine_similarity(vec_i, vec_j)

  0%|          | 0/30000 [00:00<?, ?it/s]

ValueError: ignored

In [ ]:
len(similarity_dict)

29999

In [ ]:

with open('pairwise_similarity_embedding.pkl', 'wb') as handle:
    pickle.dump(similarity_dict, handle)

In [ ]:
with open('pairwise_similarity_embedding.pkl', 'rb') as handle:
    similarity_dict = pickle.load(handle)

In [ ]:
all_similarities = np.ones((1,3))
k = 0
for i, simils in tqdm(similarity_dict.items()):
    simils = np.array(simils).reshape(-1,1)
    list_j = np.arange(i+1, len(similarity_dict)+1).reshape(-1,1)
    list_i = np.array([i]*len(list_j)).reshape(-1,1)
    similarity_ij = np.concatenate([list_i, list_j, simils], axis=1)
    all_similarities = np.concatenate([all_similarities, similarity_ij], axis=0)

    if i%1000==0:
        np.savetxt(f"all_similarities_{k}.csv", all_similarities, delimiter=",")
        all_similarities = np.ones((1,3))
        k = k + 1

  0%|          | 0/29999 [00:00<?, ?it/s]